In [1]:
import acquire, prepare, model

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [4]:
from sklearn.naive_bayes import MultinomialNB

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
import pandas as pd

In [92]:
df1 = acquire.get_corpus('data_final.json')
df2 = pd.read_json('data_second.json')
frames = [df1,df2]
df = pd.concat(frames)

In [148]:
df['prepared'] = df.readme.apply(prepare.basic_clean).apply(prepare.remove_stopwords).apply(prepare.lemmatize)

In [149]:
df = prepare.cut_singles(df)

In [150]:
df.head()

,repo,language,readme,prepared
0,freeCodeCamp/freeCodeCamp,JavaScript,\n\n\n\n\n\nfreeCodeCamp.org's open-source cod...,freecodecamporg's opensource codebase curricul...
2,vuejs/vue,JavaScript,\n\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue...,supporting vuejs vuejs mitlicensed open source...
3,facebook/react,JavaScript,\nReact · \nReact is a JavaScript library f...,react react javascript library building user i...
4,tensorflow/tensorflow,C++,\n\n\n\n\n\n\nDocumentation\n\n\n\n\n\n\n\n\nT...,documentation tensorflow endtoend open source ...
5,twbs/bootstrap,JavaScript,"\n\n\n\n\n\nBootstrap\n\n Sleek, intuitive, a...",bootstrap sleek intuitive powerful frontend fr...


In [151]:
tfidf = TfidfVectorizer()

In [152]:
tfidfs = tfidf.fit_transform(df.prepared)

In [153]:
X = pd.DataFrame(tfidfs.todense(), columns=tfidf.get_feature_names())

In [155]:
y = df.language

In [156]:
y.value_counts()

JavaScript    59
None          29
Python        21
Java          15
C++           12
Go             9
TypeScript     9
CSS            5
C              4
Ruby           3
Name: language, dtype: int64

In [195]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.25)

In [158]:
X_train.shape

(124, 34360)

In [159]:
X_test.shape

(42, 34360)

In [160]:
y_train.shape

(124,)

In [161]:
y_test.shape

(42,)

In [162]:
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

In [163]:
train.shape

(124, 1)

In [164]:
test.shape

(42, 1)

In [179]:
#THE CLASSIFIER! 5 is the best depth
tree = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train,)

In [183]:
training_predictions = tree.predict(X_train)

train['predictions'] = training_predictions

test_predictions = tree.predict(X_test)

test['predictions'] = test_predictions

In [184]:
model.score_your_model(train.actual, train.predictions)

Accuracy: 82.26%
---
Confusion Matrix
actual       C  C++  CSS  Go  Java  JavaScript  None  Python  Ruby  TypeScript
predictions                                                                   
C++          1    9    2   0     1           3     4       2     0           3
CSS          0    0    2   0     0           0     0       0     0           0
Go           2    0    0   7     0           0     0       1     2           0
Java         0    0    0   0    10           0     0       0     0           0
JavaScript   0    0    0   0     0          41     0       0     0           1
None         0    0    0   0     0           0    17       0     0           0
Python       0    0    0   0     0           0     0      13     0           0
TypeScript   0    0    0   0     0           0     0       0     0           3
---
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
         C++       0.36      1.00      0.53         9
     

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [185]:
model.score_your_model(test.actual, test.predictions)

Accuracy: 50.00%
---
Confusion Matrix
actual       C  C++  CSS  Go  Java  JavaScript  None  Python  Ruby  TypeScript
predictions                                                                   
C++          0    1    0   1     1           1     2       2     1           0
CSS          0    0    0   0     0           0     1       0     0           0
Go           1    1    0   1     0           0     0       0     0           1
Java         0    0    0   0     3           0     1       0     0           0
JavaScript   0    1    1   0     0          11     1       1     0           0
None         0    0    0   0     0           1     2       0     0           0
Python       0    0    0   0     0           1     1       2     0           0
TypeScript   0    0    0   0     0           1     0       0     0           1
---
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         1
         C++       0.11      0.33      0.17         3
     

In [142]:
def make_model_components(feature_variable, target_variable, test_size=.25):
    """
    Takes the one column as the feature matrix, one column as target variable. Splits them into test-train.
    Creates a dataframe for the Test and Train of the target variables (y_train, y_test)
    """
    tfidf = TfidfVectorizer()
    tfidfs = tfidf.fit_transform(feature_variable.values)
    X = pd.DataFrame(tfidfs.todense(), columns=tfidf.get_feature_names())
    y = target_variable
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=test_size)
    train = pd.DataFrame(dict(actual=y_train))
    test = pd.DataFrame(dict(actual=y_test))
    return X_train, X_test, train, test

In [143]:
X_train, X_test, train, test = make_model_components(df.prepared, df.language)

In [144]:
X_train.shape

(124, 19319)

In [145]:
train.shape

(124, 1)

In [146]:
X_test.shape

(42, 19319)

In [147]:
test.shape

(42, 1)

In [ ]:
#tfidf = TfidfVectorizer()
#tfidfs = tfidf.fit_transform(feature_variable.values)
#X = pd.DataFrame(tfidfs.todense(), columns=tfidf.get_feature_names())
#y = target_variable
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=test_size)
#train = pd.DataFrame(dict(actual=y_train))
#test = pd.DataFrame(dict(actual=y_test))
#return X_train, X_test, train, test

In [ ]:
# def run_the_tree_model(X,y,depth=7):
#     """
#     We'll have to pick some classification models to use on the train-test sets.
#     """
#     #tree = DecisionTreeClassifier(max_depth=depth).fit(X, y)
#     predictions = tree.predict(X)
#     return predictions